In [1]:
import genetic
from genetic import utils, error_handling
import copy

In [2]:
genetic.random.seed(0)

In [3]:
class Gene:
    recessive = False
    mutable = False
    groups = []
    value = None

    def __init__(self, value, mutable = False):
        self.mutable = mutable
        self.value = value

    def __add__(self, other):
        if type(other) == type(self):
            return GeneGroup(self, other)
        elif type(other) == GeneGroup:
            return other + self

    def __repr__(self):
        return "Gene(%s)" % str(self.value)

In [4]:
class DNA:
    def __init__(self):
        self.geneGroups = []
        self.genes = []
        self.maxGenes = False

        self._groups = {}
        self._index = -1

    def __iter__(self):
        return self

    def __next__(self):
        self._index += 1
        if self._index > len(self) - 1:
            raise StopIteration
        return self.values[self._index]

    def __len__(self):
        return len(self.genes)

    def __getitem__(self, name):
        if type(name) == str:
            if name in self._groups:
                return self._groups[name]
            raise KeyError("Group not found")
        else:
            raise AttributeError("Object only accepts string as index")
    
    def __setitem__(self, name, value):
        if type(name) == str:
            self._groups[name] = value
        else:
            raise AttributeError("Object only accepts string as a key")
    
    def __add__(self, other):
        if type(other) == Gene:
            return copy.deepcopy(self.newGene(other))
        elif type(other) == type(self):
            self.genes += other.genes
            return self
        else:
            raise NotImplemented
    
    def __iadd__(self, other):
        if type(other) == GeneGroup:
            self.genes += other.genes
            return self
        else:
            raise NotImplemented

    def _chooseRandomGene(self, genesList, numberOfGenes = 0 , numberOfGroups = 0):
        if numberOfGenes == 0 and numberOfGroups == 0:
            raise AttributeError("numberOfGenes or numberOfGroups can be zero, not both")
        
        totalGenes = []
        if numberOfGenes > 0:
            totalGenes += [genes for genes in genesList if type(genes) == Gene]
        if numberOfGroups > 0:
            totalGenes += [genes for genes in genesList if type(genes) == GeneGroup]

        return utils.globalChoice(
            totalGenes,
            numberOfItemsToGet = numberOfGenes,
            allowRepeats = True
        )

    def get(self, name, defaultReturn = False):
        if name in self._groups:
            return self._groups[name]
        return defaultReturn

    def setGenes(self, *genes):
        if len(genes) > 0:
            self.genes = []
            for gene in genes:
                self.newGene(gene)
        else:
            raise error_handling.Genes("Gene List can't be empty")

    def generateGeneSequence(self, possibleGenes, maxGenes=False):
        if maxGenes:
            self.maxGenes = maxGenes
        if self.maxGenes:
            self.genes = self._chooseRandomGene(genesList = possibleGenes, numberOfGenes = self.maxGenes)
        for genes in possibleGenes:
            if type(genes) == GeneGroup:
                genes.generateGeneSequence(genes.genes)
                self.newGene(genes)

    def newGene(self, *values, mutable = False):
        if len(values) == 0:
            raise error_handling.Genes("Gene List can't be empty")
        for newValues in values:
            newValues = copy.deepcopy(newValues)
            if type(newValues) == GeneGroup:
                if not self.get(newValues.name, False):
                    if self not in newValues.geneGroups:
                        newValues.geneGroups.append(self)

                    self[newValues.name] = newValues
                    self.genes.append(newValues)
                else:
                    self[newValues.name] += newValues


            elif type(newValues) == Gene:
                if self not in newValues.groups:
                    newValues.groups.append(self)
                self.genes.append(newValues)

            else:
                newValues = Gene(newValues)
                self.genes.append(newValues)
        
        return self

In [5]:
class GeneGroup(DNA):
    def __init__(self, *values, name="", mutable = False, maxGenes = False):
        DNA.__init__(self)

        self.name = name
        self.mutable = mutable
        self.maxGenes = maxGenes

        if len(values) > 0:
            self.setGenes(*values)

    def __repr__(self):
        return "Group: %s(%s)" % (self.name, self.genes)

In [6]:
groupA = GeneGroup(8, 9)
groupB = GeneGroup(30, 50)

print(groupA + groupB)

Group: ([Gene(8), Gene(9), Gene(30), Gene(50)])


In [7]:
class Specie(DNA):
    fitness = genetic.NINF
    age = 0
    primordial = True
    child = False
    conditionsMet = False
    parents = ()

    def __init__(self, *genes, name = ""):
        DNA.__init__(self)
        self.name = name

        if len(genes) > 0:
            self.setGenes(*genes)

    def __repr__(self):
        return "Specie: %s(%s)" % (self.name, self.genes)

    def createName(self, namesOptions):
        self.speciesName = "%s.%d" % (genetic.random.choice(namesOptions), genetic.random.randint(1000))
        return self.speciesName

In [13]:
SpecieA = Specie(10, 40, 80, name="Mafagafo")
SpecieB = Specie(30, 10, name="WeirdStuff")

print(SpecieA + SpecieB)

Specie: Mafagafo([Gene(10), Gene(40), Gene(80), Gene(30), Gene(10)])


In [9]:
class Population(DNA):
    _allPopulation = []

    historic = []
    defaultSpeciesNumber = False
    better = Specie()
    
    def __init__(self, defaultSpeciesCount = False, specieObject = Specie):
        DNA.__init__(self)
        self._allPopulation = []
        self.specieObject = specieObject

        self.historic = []
        self.defaultSpeciesNumber = defaultSpeciesCount

    def __getitem__(self, name):
        if type(name) == int:
            return self._allPopulation[name]
        elif type(name) == str:
            return DNA.__getitem__(self, name)
        else:
            raise AttributeError("Use int to get specie from population or string to get avaiable GeneGroup")

    def __setitem__(self, name, value):
        if type(name) == int:
            self._allPopulation[name] = value
        elif type(name) == str:
            DNA.__setitem__(self, name, value)
        else:
            raise AttributeError("Use int to define specie from population or string to define value of GeneGroup")

    def __iter__(self):
        return self

    def __repr__(self):
        return "Population: %s" % self._allPopulation

    def __next__(self):
        self._index += 1
        if self._index > len(self._allPopulation) - 1:
            raise StopIteration
        return self._allPopulation[self._index]

    def addToPopulation(newSpecie):
        if newSpecie.maxGenes:
            if len(newSpecie) < newSpecie.maxGenes
                newSpecie.generateGeneSequence(newSpecie.genes, maxGenes = genesPerSpecie, allowRepeat = True)
            elif len(newSpecie) > newSpecie.maxGenes
                newSpecie.generateGeneSequence(newSpecie.genes, maxGenes = genesPerSpecie, allowRepeat = False)

        self._allPopulation.append(newSpecie)

    def generatePopulation(self, numberOfSpecies, genesPerSpecie, useRecessives = True, reset = True):
        if reset:
            self._allPopulation = []
        for newSpecie in range(numberOfSpecies):
            newSpecie = self.specieObject()
            newSpecie.maxGenes = genesPerSpecie
            newSpecie.generateGeneSequence(self.genes, maxGenes = genesPerSpecie)
            self._allPopulation.append(newSpecie)

In [10]:
W = 119
A = 97
S = 115
D = 100

possibleGenes = [
    GeneGroup(10, 5, 2, 4, name="weights", maxGenes=2),
    [W, W],
    [A, A],
    [S, S],
    [D, D],

    [W, D],
    [W, A],
    [S, D],
    [S, A]
]

In [11]:
newPopulation = Population()
[newPopulation.newGene(gene) for gene in possibleGenes]
newPopulation.generatePopulation(5, 2)
for specie in newPopulation:
    print(specie)

Specie: ([Gene([119, 100]), Gene([119, 97]), Group: weights([Gene(10), Gene(5)])])
Specie: ([Gene([100, 100]), Gene([119, 97]), Group: weights([Gene(10), Gene(10)])])
Specie: ([Gene([115, 115]), Gene([119, 100]), Group: weights([Gene(10), Gene(10)])])
Specie: ([Gene([115, 100]), Gene([119, 119]), Group: weights([Gene(10), Gene(10)])])
Specie: ([Gene([119, 119]), Gene([119, 100]), Group: weights([Gene(10), Gene(10)])])
